In [2]:
import csv
import random
import re
from IPython.display import display, HTML
import requests
from datetime import datetime

# === File path ===
DATASET_FILE = "Unstyled1.csv"
SUGGESTION_LOG = "last_suggestion.txt"

# === API Setup ===
WEATHER_API_KEY = "e7b15095fe3450a04772a31d67478817"   # Replace with your OpenWeatherMap API key

# Step 1: Load + Clean dataset
def load_dataset():
    outfits = []
    with open(DATASET_FILE, newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            normalized_row = {}
            for k, v in row.items():
                if not v:
                    continue
                key = k.strip().lower()
                value = v.strip()

                # Extract URL from Excel HYPERLINK formula
                if value.startswith("=HYPERLINK"):
                    match = re.search(r'\"(https?://[^\"]+)\"', value)
                    if match:
                        value = match.group(1)

                # Keep only required columns
                if key in ["event", "season", "topwear", "bottomwear", "footwear", "accessories", "gender"]:
                    normalized_row[key] = value

            outfits.append(normalized_row)
    return outfits

# Step 2: Get user's location (IP-based)
def get_location():
    try:
        response = requests.get("https://ipinfo.io")
        data = response.json()
        return data.get("city")
    except:
        return None

# Step 3: Get current weather/season
def get_season_from_weather(city):
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
        response = requests.get(url)
        data = response.json()
        temp = data['main']['temp']

        if temp >= 25:
            return "summer"
        elif temp <= 15:
            return "winter"
        else:
            return "rainy"
    except:
        return None

# Step 4: Find matching outfits (event + season)
def find_outfits(event, season):
    dataset = load_dataset()
    matches = []
    for row in dataset:
        row_event = row.get("event", "").strip().lower()
        row_season = row.get("season", "").strip().lower()
        if row_event == event.strip().lower() and row_season == season.strip().lower():
            matches.append(row)
    return matches

# Step 5: Show outfits (display images)
def show_outfits(outfit_row):
    html_images = ""
    for key in ["topwear", "bottomwear", "footwear", "accessories"]:
        link = outfit_row.get(key, "").strip()
        if link:
            html_images += f"""
            <div style="display:inline-block;margin:10px;text-align:center;">
                <img src="{link}" style="width:200px;height:200px;object-fit:cover;border-radius:12px;">
                <div>{key.capitalize()}</div>
            </div>
            """
    display(HTML(html_images))

# Step 6: Daily restriction
def already_suggested_today():
    today_str = datetime.today().strftime("%Y-%m-%d")
    try:
        with open(SUGGESTION_LOG, "r") as f:
            last_date = f.read().strip()
        return last_date == today_str
    except FileNotFoundError:
        return False

def mark_suggestion_today():
    today_str = datetime.today().strftime("%Y-%m-%d")
    with open(SUGGESTION_LOG, "w") as f:
        f.write(today_str)

# === Main Logic ===
print("\n Outfit Recommendation Based on Live Weather \n")

event = input("Enter your event (office, trip, marriage, party etc.): ").strip().lower()

city = get_location()
if not city:
    print("Could not determine location. Using default season 'summer'.")
    season = "summer"
else:
    season = get_season_from_weather(city)
    if not season:
        print("Could not fetch weather. Using default season 'summer'.")
        season = "summer"
    else:
        print(f"Detected location: {city}, Season based on temperature: {season}")

outfits = find_outfits(event, season)

if not outfits:
    print(f"No outfits found for event '{event}' in season '{season}'.")
else:
    if already_suggested_today():
        print("""

Dear user,
You have already received your free outfit suggestion today.
Upgrade to Premium for unlimited daily suggestions.
""")
    else:
        chosen_outfit = random.choice(outfits)
        print(f"Outfit suggestions for '{event}' in '{season}':")
        show_outfits(chosen_outfit)
        mark_suggestion_today()



 Outfit Recommendation Based on Live Weather 

Enter your event (office, trip, marriage, party etc.): office
Detected location: Council Bluffs, Season based on temperature: winter


Dear user,
You have already received your free outfit suggestion today.
Upgrade to Premium for unlimited daily suggestions. 

